This notebook walks through building a simple deep learning model (Convolutional Neural Network) that can automatically classify images of honey bees vs images of bumble bees and then loads a pre-trained model for evaluation.

In [9]:
## Import Python libraries

import pickle
from pathlib import Path
from skimage import io

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import keras

from keras.models import Sequential

from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

In [15]:
## Load image labels

# Change directory to point to data. 
%cd C:\Users\scheid\Dropbox\Data Science\Deep Learning Projects\Naive Bees_ Deep Learning with Images\datasets

# load labels.csv from datasets folder using pandas
labels = pd.read_csv('labels.csv', index_col=0)

# print value counts for genus
print(labels.genus.values)

# assign the genus label values to y
y = labels.genus.values

C:\Users\scheid\Dropbox\Data Science\Deep Learning Projects\Naive Bees_ Deep Learning with Images\datasets
[1. 1. 1. ... 0. 0. 0.]


In [16]:
cd

C:\Users\scheid


In [11]:
## Scale our data so that it has a mean of 0 and standard deviation of 1.

# initialize standard scaler
ss = StandardScaler()

image_list = []
for i in labels.index:
    # load image
    img = io.imread('{}.jpg'.format(i)).astype(np.float64)
    
    # for each channel, apply standard scaler's fit_transform method
    for channel in range(img.shape[2]):
        img[:, :, channel] = ss.fit_transform(img[:, :, channel])
        
    # append to list of all images
    image_list.append(img)
    
# convert image list to single array
X = np.array(image_list)

# print shape of X
print(np.size(X))

FileNotFoundError: No such file: 'C:\Users\scheid\Dropbox\Data Science\Deep Learning Projects\Naive Bees_ Deep Learning with Images\datasets\520.jpg'

In [ ]:
## Split into train, test and evaluation sets

# split out evaluation sets (x_eval and y_eval)
x_interim, x_eval, y_interim, y_eval = train_test_split(X,
                                           y,
                                           test_size=0.2,
                                           random_state=52)

# split remaining data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x_interim,
                                           y_interim,
                                           test_size=0.4,
                                           random_state=52)


# examine number of samples in train, test, and validation sets
print('x_train shape:', x_train.shape[:])
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_eval.shape[0], 'eval samples')

In [ ]:
## Begin building model

# set model constants
num_classes = 1

# define model as Sequential
model = Sequential()

# first convolutional layer with 32 filters
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(50, 50, 3)))

# add a second 2D convolutional layer with 64 filters
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(50, 50, 3)))
    

In [ ]:
## Continue building model

# reduce dimensionality through max pooling
model.add(MaxPooling2D(pool_size=(2,2)))

# third convolutional layer with 64 filters
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
# add dropout to prevent over fitting
model.add(Dropout(0.25))
# necessary flatten step preceeding dense layer
model.add(Flatten())
# fully connected layer
model.add(Dense(128, activation='relu'))

# add additional dropout to prevent overfitting
model.add(Dropout(0.5))

# prediction layers
model.add(Dense(num_classes, activation='sigmoid', name='preds'))

# show model summary
model.summary()

In [ ]:
## Compile and train model

model.compile(
    # set the loss as binary_crossentropy
    loss='binary_crossentropy',
    # set the optimizer as stochastic gradient descent
    optimizer=keras.optimizers.SGD(lr=0.001),
    # set the metric as accuracy
    metrics=['accuracy']
)

# mock-train the model using the first ten observations of the train and test sets
model.fit(
    x_train[:10, :, :, :],
    y_train[:10],
    epochs=5,
    verbose=1,
    validation_data=(x_test[:10, :, :, :], y_test[:10])
)

In [ ]:
## Load pre-trained and look at training history

# Change directory to point to data. 
%cd C:\Users\scheid\Dropbox\Data Science\Deep Learning Projects\Naive Bees_ Deep Learning with Images\datasets

# load pre-trained model
pretrained_cnn = keras.models.load_model('pretrained_model.h5')

# evaluate model on test set
score = pretrained_cnn.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

print("")

# evaluate model on holdout set
eval_score = pretrained_cnn.evaluate(x_eval, y_eval, verbose=0)
# print loss score
print('Eval loss:', eval_score[0])
# print accuracy score
print('Eval accuracy:', eval_score[1])

In [14]:
## Generate predictions

# predicted probabilities for x_eval
y_proba = pretrained_cnn.predict(x_eval)

print("First five probabilities:")
print(y_proba[:5])
print("")

# predicted classes for x_eval
y_pred = pretrained_cnn.predict_classes(x_eval)

print("First five class predictions:")
print(y_pred[:5])
print("")

C:\Users\scheid
